In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.ticker as mtick
import matplotlib.cm as cm
import math
import plotly.graph_objects as go
from collections import OrderedDict
from matplotlib.lines import Line2D  # Import for creating custom legend markers
import json
import copy
import random
import re
import seaborn as sns
import importlib


# Import BW25 packages
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa
import brightway2 as bw 
from bw2calc import LeastSquaresLCA

In [2]:
from  standards import *
import Monte_Carlo as MC
import life_cycle_assessment as lc
import LCA_plots as lp


importlib.reload(MC)
importlib.reload(lc)
importlib.reload(lp)

Folder name Results created
Folder name Results created


<module 'LCA_plots' from 'c:\\Users\\ruw\\Desktop\\RA\\Single-use-vs-multi-use-in-health-care\\Brighway\\LCA_plots.py'>

In [3]:
db_type = 'CONSQ'
lcia_method = 'recipe'

save_dir = results_folder('Results')

file_name = f'Results\Ananas - {db_type}_{lcia_method}.xlsx'
file_cont_name = f'Results\Ananas contribution - {db_type}_{lcia_method}.xlsx'
sheet_name = 'Ananas'

path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Brighway'

# Your name variable should be the name of the project.
database_project = "RA - Ofir & Stine 1"
database_name = "Sterilization"

flows = [f'alubox - {db_type}', f'PP sheet sc2 - {db_type}', f'PP sheet sc3 - {db_type}']
flow_legend = [f'SAL', f'LAL', f'PPIC', f'PPREC']

directory = 'C:\\Users\\ruw\\Desktop\\RA\\'

calculate_lcia = False
calculate_cont = False

Folder name Results created


In [4]:
bd.projects.set_current(database_project)
db = bw.Database(database_name)

bd.databases
db

Brightway2 SQLiteBackend: Sterilization

In [5]:
old_consq = 'Consq EcoInvent'
old_apos = 'APOS EcoInevnt'
oconsq = bw.Database(old_consq)

In [6]:
# Get processes
processes = [act for act in db]
for proc in processes:
    print(f'process: {proc}')
    for exc in proc.exchanges():
        if exc['type'] == 'technosphere' or  exc['type'] == 'biosphere':
            exc_sub = exc.input
            print(f"--{exc.input}, {exc['amount']}---")
            # for x in exc_sub.exchanges():
            #     xin = x.input
            #     if exc['type'] == 'technosphere' or  exc['type'] == 'biosphere':
            #         print(xin, xin[0], xin[1], x['amount'])
            # print()

process: 'alubox small + washer - CONSQ' (unit, GLO, None)
--'alubox EoL melting - CONSQ' (kilogram, GLO, None), -0.004641---
--'alubox EoL mixed sorting - CONSQ' (kilogram, GLO, None), -0.004641---
--'alubox production - CONSQ' (kilogram, GLO, None), 0.004641---
--'alubox raw materials - CONSQ' (kilogram, GLO, None), 0.004641---
--'autoclave - CONSQ' (unit, GLO, None), 0.1111111111111111---
--'avoided alubox raw materials - CONSQ' (kilogram, GLO, None), -0.00440895---
--'cabinet washer - CONSQ' (unit, GLO, None), 1---
--'market for polysulfone' (kilogram, GLO, None), 0.000162---
--'transport Alu - CONSQ' (ton kilometer, GLO, None), 0.00538546193747744---
process: 'H400 REC - APOS' (unit, GLO, None)
--'PE incineration no Energy Recovery - APOS' (kilogram, CH, None), -0.01009237875288686---
--'PP incineration no Energy Recovery - APOS' (kilogram, CH, None), -0.0874---
--'autoclave - APOS' (unit, GLO, None), 0.1111111111111111---
--'energy avoided electricity from energy recovery - APOS'

In [7]:
new_consq = 'ev391consq'
new_apos = 'ev391apos'
bs3 = 'biosphere3'

In [8]:
consq_path = r'C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets'

database_new = 'SU_vs_MU'

# Create a new database for your extracted processes



bd.projects.set_current(database_new)
new_database = bw.Database("sterilization")
if bs3 in bd.databases:
    print('biosphere3 is present')
else:
    bi.bw2setup()
    
if new_consq in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    ei = bi.SingleOutputEcospold2Importer(dirpath=consq_path, db_name='ev391consq') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

biosphere3 is present
Ecoinvent 3.9.1 is already present in the project.


In [9]:
apos_path = r'C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets'

if new_apos in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    ei = bi.SingleOutputEcospold2Importer(dirpath=apos_path, db_name='ev391apos') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

Ecoinvent 3.9.1 is already present in the project.


In [10]:
# new_data = {}
# for act in processes:
#     code = act[1]
#     new_data[code] = act._data
#     # print(act)
#     for exc in act.exchanges():
#         if exc['type'] == 'technosphere' and exc['input'][0] == old_consq:
#             exc['input'] = (new_consq, exc['input'][1])
#             print(exc['input'])
#     new_data[(new_database.name, act_dict['code'])] = act_dict

# new_database.write(new_data)

In [11]:
# Create and set the new project
bw.projects.set_current(database_new)

# Import the new Ecoinvent database into the new project
ecoinvent_new = bw.Database(new_consq)
# ecoinvent_new.import_data("path_to_ecoinvent_new_data")

# Set the current project to the existing one
bw.projects.set_current(database_project)

# Access the existing processes
old_database = bw.Database(old_consq)
processes = [act for act in old_database]

# Switch back to the new project
bw.projects.set_current(new_database)

# Create a new database for your extracted processes
new_database = bw.Database('sterilization')

# Function to create new process
def create_new_process_with_dependencies(old_process, new_database, ecoinvent_new):
    new_process_data = old_process.as_dict()
    new_process_data['database'] = new_database.name  # Update the database name
    new_process_key = (new_database.name, new_process_data['code'])
    
    if 'exchanges' in new_process_data:
        for exc in new_process_data['exchanges']:
            if exc['type'] == 'technosphere' or exc['type'] == 'biosphere':
                exc_sub = exc.input
                if exc['type'] == 'technosphere' and exc['input'][0] == 'Consq EcoInvent':
                    exc['input'] = (ecoinvent_new.name, exc['input'][1])  # Update to new ecoinvent
                print(f"--{exc['input']}, {exc['amount']}---")
                
                for x in exc_sub.exchanges():
                    xin = x.input
                    if x['type'] == 'technosphere' or x['type'] == 'biosphere':
                        print(xin, xin[0], xin[1], x['amount'])

    new_database.write({new_process_key: new_process_data})
    
# Create and write new processes in the new database
for proc in processes:
    print(f'Creating new process: {proc}')
    create_new_process_with_dependencies(proc, new_database, ecoinvent_new)